# Показатели ТТВ по нескольким ЦА
Пример расчета показателей Тотал ТВ.

## Описание задачи и условий расчета

- Период: сентябрь 2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+, Мужчины, Женщины
- Место просмотра: Все места (дом+дача)
- Каналы: Total Channels									
- Статистики: Universe, TTV Rtg%, TTV AvReach%, TTV Reach%, TTV ATV

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['пол', 'возраст'])

Запрошены записи: 0 - 129
Всего найдено записей: 129



,id,valueId,name,colName,valueName
0,1,1,Пол,Sex,Мужчины
1,1,2,Пол,Sex,Женщины
2,26,1,Семейное положение,MaritalStatus,Женат / замужем
3,26,2,Семейное положение,MaritalStatus,Холост / не замужем
4,2,33,Возраст,Age,33
...,...,...,...,...,...
124,2,10,Возраст,Age,10
125,2,89,Возраст,Age,89
126,2,8,Возраст,Age,8
127,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


## Формирование задания
Зададим условия расчета

In [3]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-09-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter=None

# Указываем список статистик для расчета
statistics = ['Universe000', 'TTVRtgPer', 'TTVAvReachPer', 'TTVReachPer', 'TTVATV']

# Указываем срезы
slices = []

# Задаем опции расчета
# В данном примере для расчета показателей Total Channels можно не указывать "totalType", так как по умолчанию 
# будет рассчитываться именно он
# Для расчета показателей Total Tv Set в options ниже необходимо добавить "totalType": "TotalTVSet"
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группу значений параметров для различных ЦА

In [4]:
targets = {
    'Total Ind.':'age >= 4',
    'Male 4+':'age >= 4 AND sex = 1',
    'Female 4+':'age >= 4 AND sex = 2'    
}

## Расчет задания

In [5]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задания и отправляем на расчет
for target, logical in targets.items():
    
    # Формируем задание на расчет 
    project_name = f"{target}" 
    basedemo_filter = f"{logical}"
      
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)
df

Отправляем задания на расчет
...
Ждем выполнения
Расчет задач (3) [ = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = ] время расчета: 0:02:02.469861
Расчет завершен, получаем результат
Собираем таблицу
...

,prj_name,Universe000,TTVReachPer,TTVRtgPer,TTVAvReachPer,TTVATV
0,Total Ind.,136595.065986,97.797383,14.467334,64.701274,217.749093
0,Male 4+,63089.141504,97.341193,12.780146,61.481023,192.179164
0,Female 4+,73505.924482,98.188927,15.915424,67.465172,239.695418


## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [6]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('02_TTV_figures'))
df_info = mtask.task_builder.get_report_info()
df.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()